In [1]:
import pandas as pd

In [8]:
df = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/us-states.csv",
                parse_dates=["date"])

In [9]:
df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
61937,2023-03-23,Virginia,51,2298300,23782
61938,2023-03-23,Washington,53,1940704,15905
61939,2023-03-23,West Virginia,54,645710,8132
61940,2023-03-23,Wisconsin,55,2014524,16485


In [10]:
df.dtypes

date      datetime64[ns]
state             object
fips               int64
cases              int64
deaths             int64
dtype: object

In [11]:
cases_by_date = df.set_index("date")

In [13]:
cases_by_date.iloc[10]

state     Illinois
fips            17
cases            1
deaths           0
Name: 2020-01-26 00:00:00, dtype: object

In [19]:
cases_by_date.loc["January 01, 2022":"February 28, 2023"]

,state,fips,cases,deaths
date,,,,
2022-01-01,Alabama,1,896614,16455
2022-01-01,Alaska,2,156130,954
2022-01-01,American Samoa,60,11,0
2022-01-01,Arizona,4,1389708,24354
2022-01-01,Arkansas,5,570641,9180
...,...,...,...,...
2023-02-28,Virginia,51,2287440,23738
2023-02-28,Washington,53,1924418,15668
2023-02-28,West Virginia,54,640150,8048


In [24]:
us_cases_by_date = cases_by_date.groupby("date")["cases"].sum()
us_cases_by_date

date
2020-01-21            1
2020-01-22            1
2020-01-23            1
2020-01-24            2
2020-01-25            3
                ...    
2023-03-19    103784916
2023-03-20    103794703
2023-03-21    103813076
2023-03-22    103864888
2023-03-23    103910034
Name: cases, Length: 1158, dtype: int64

In [23]:
import bqplot

In [26]:
us_cases_by_date.values

array([        1,         1,         1, ..., 103813076, 103864888,
       103910034])

In [32]:
import numpy as np

In [38]:
x_sc = bqplot.DateScale()
y_sc = bqplot.LinearScale()

x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, orientation = 'vertical')

pan_zoom = bqplot.interacts.PanZoom(scales = {'x': [x_sc], 'y': [y_sc]})

line = bqplot.Lines(x = us_cases_by_date.index, y = us_cases_by_date.values, scales = {'x': x_sc, 'y': y_sc})
figure = bqplot.Figure(marks = [line], axes = [x_ax, y_ax], interaction = pan_zoom)
display(figure)

Figure(axes=[Axis(scale=DateScale()), Axis(orientation='vertical', scale=LinearScale())], fig_margin={'top': 6…

In [39]:
us_state_map = bqplot.topo_load("map_data/USStatesMap.json")

In [45]:
proj_scale = bqplot.AlbersUSA()
color_scale = bqplot.ColorScale()

color_axis = bqplot.ColorAxis(scale = color_scale)

map_mark = bqplot.Map(map_data = us_state_map,
                      color = cases_by_date.groupby("fips")["cases"].max().to_dict(),
                      scales = {'projection': proj_scale, 'color': color_scale})

figure = bqplot.Figure(marks = [map_mark], axes = [color_axis])
display(figure)

Figure(axes=[ColorAxis(scale=ColorScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, ma…

In [46]:
import ipywidgets

In [74]:
x_sc = bqplot.DateScale()
y_sc = bqplot.LinearScale()

x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, orientation = 'vertical')

pan_zoom = bqplot.interacts.PanZoom(scales = {'x': [x_sc], 'y': [y_sc]})

line = bqplot.Lines(x = us_cases_by_date.index, y = us_cases_by_date.values, scales = {'x': x_sc, 'y': y_sc})
line_figure = bqplot.Figure(marks = [line], axes = [x_ax, y_ax], interaction = pan_zoom)

proj_scale = bqplot.AlbersUSA()
color_scale = bqplot.ColorScale()

color_axis = bqplot.ColorAxis(scale = color_scale)

map_mark = bqplot.Map(map_data = us_state_map,
                      color = cases_by_date.groupby("fips")["cases"].max().to_dict(),
                      scales = {'projection': proj_scale, 'color': color_scale})

map_figure = bqplot.Figure(marks = [map_mark], axes = [color_axis])

date_picker = ipywidgets.DatePicker()

def on_pick_date(change):
    new_date = pd.to_datetime(change['new'])
    map_mark.color = cases_by_date.loc[new_date].groupby("fips")["cases"].max().to_dict()

date_picker.observe(on_pick_date, ["value"])

hb = ipywidgets.VBox([date_picker, ipywidgets.HBox([line_figure, map_figure])])

map_figure.layout.width = '50%'
line_figure.layout.width = '50%'
hb.layout.width = '100%'
hb.layout.height = '100%'
hb

In [95]:
x_sc = bqplot.DateScale()
y_sc = bqplot.LinearScale()

x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, orientation = 'vertical')

fi_sel = bqplot.interacts.FastIntervalSelector(scale = x_sc)

line = bqplot.Lines(x = us_cases_by_date.index, y = us_cases_by_date.values, scales = {'x': x_sc, 'y': y_sc})
line_figure = bqplot.Figure(marks = [line], axes = [x_ax, y_ax], interaction = fi_sel)

def change_selection(change):
    if change['new'] is not None and change['new'].size > 0:
        start, stop = change['new']
        values = cases_by_date.loc[pd.to_datetime(start):pd.to_datetime(stop)].groupby("fips")
        v1 = values["cases"].min()
        v2 = values["cases"].max()
        delta_cases.value = f"Difference of {(v2 - v1).sum():0.2e}"
        map_mark.color = (v2 - v1).to_dict()

delta_cases = ipywidgets.Label()

fi_sel.observe(change_selection, "selected")

proj_scale = bqplot.AlbersUSA()
color_scale = bqplot.ColorScale()

color_axis = bqplot.ColorAxis(scale = color_scale)

map_mark = bqplot.Map(map_data = us_state_map,
                      color = cases_by_date.groupby("fips")["cases"].max().to_dict(),
                      scales = {'projection': proj_scale, 'color': color_scale})

map_figure = bqplot.Figure(marks = [map_mark], axes = [color_axis])

hb = ipywidgets.VBox([delta_cases, ipywidgets.HBox([line_figure, map_figure])])

map_figure.layout.width = '50%'
line_figure.layout.width = '50%'
hb.layout.width = '100%'
hb.layout.height = '100%'
hb

In [94]:
fi_sel.selected